In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import cross_val_score
import pandas as pd

In [2]:
dset_trn = pd.read_csv('metadata.csv')
dset_tst = pd.read_csv('metadata_test.csv')

In [3]:
X_train = dset_trn.drop(columns=['user_snippet','activity']).to_numpy()
X_test = dset_tst.drop(columns='user_snippet').to_numpy()
y_train = dset_trn['activity'].to_numpy()

In [5]:


pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('feature_selection', SelectFromModel(RandomForestClassifier())),
    ('rf', RandomForestClassifier())
])


In [4]:
from sklearn import  preprocessing 

mm_scaler = preprocessing.StandardScaler()  # creates a Transformer (set of preprocessing operations)
X_train = mm_scaler.fit_transform(X_train)  # estimates transformation parameters using the training set, and applies to them
X_test = mm_scaler.transform(X_test) 

In [6]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'rf__n_estimators': [100, 200, 500],
    'rf__max_depth': [10, 20, 30],
    'rf__min_samples_split': [2, 5, 10],
    'feature_selection__threshold': ['mean', 'median', '1.25*mean']
}

In [7]:
# Define the cross-validation strategy
cv = 5

In [8]:
# Define the scoring metric
#scorer = make_scorer(accuracy_score)

In [9]:
# Perform hyperparameter tuning using grid search
grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=cv, scoring=scorer, n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('feature_selection',
                                        SelectFromModel(estimator=RandomForestClassifier())),
                                       ('rf', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'feature_selection__threshold': ['mean', 'median',
                                                          '1.25*mean'],
                         'rf__max_depth': [10, 20, 30],
                         'rf__min_samples_split': [2, 5, 10],
                         'rf__n_estimators': [100, 200, 500]},
             scoring=make_scorer(accuracy_score))

In [10]:
y_tst_gs1 = grid_search.predict(X_test)

dset_sol_gs1 = pd.DataFrame({"user_snippet": dset_tst.user_snippet, "Prediction" : y_tst_gs1})
dset_sol_gs1.head()

,user_snippet,Prediction
0,2066_0,Upstairs
1,2066_1,Walking
2,2066_2,Jogging
3,2066_3,Walking
4,2066_4,Walking


In [11]:
dset_sol_gs1.to_csv("activityprediction_rf3.csv", index=False)